# Projeto Aplicado IV — Etapa 3  
**EDA + Modelo Base (SARIMAX) e validação base**

> Este notebook está estruturado para cumprir os requisitos da *Etapa 3*: EDA, pré-processamento,
> modelo base com validação (holdout + CV), diagnósticos, e comparação com baseline(s).
> A linguagem é impessoal e as decisões são justificadas pela EDA.

In [ ]:
# =======================
# Imports e configuração
# =======================
import warnings, io, os, sys, math
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.stats.stattools import jarque_bera
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Warnings controlados
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore",
    message="Non-invertible starting MA parameters found",
    category=UserWarning, module="statsmodels"
)

print("Versões -> python:", sys.version.split()[0],
      "| numpy:", np.__version__,
      "| pandas:", pd.__version__)

## Dados e construção do `df_model`
- Frequência **Q-DEC** (trimestral, fim do trimestre).
- Série-alvo: `tx_desocupacao`.
- Exógenas candidatas: `caged_saldo_tri`, `selic_tri` (ablação).
- Dummies de trimestre para ML (`Q_1..Q_4`) e `post_2021` (mudança de regime).  
- `d_covid`: dummy de choque estrutural 2020Q2–2021Q1.

In [ ]:
# =======================
# Carregar CSV e recriar df_model
# =======================
import requests

CSV_CANDIDATES = [
    Path("/mnt/data/caged_pnad_selic_trimestral.csv"),
    Path("dataset/tratados/caged_pnad_selic_trimestral.csv")
]
RAW_URL = ("https://raw.githubusercontent.com/"
           "fpaterni10/projeto-aplicado-iv-desemprego-br/main/"
           "dataset/tratados/caged_pnad_selic_trimestral.csv")

csv_path = None
for p in CSV_CANDIDATES:
    if p.exists():
        csv_path = p
        break

if csv_path is not None:
    df0 = pd.read_csv(csv_path)
else:
    # fallback: tentar baixar (caso rode fora deste ambiente)
    try:
        r = requests.get(RAW_URL, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        df0 = pd.read_csv(io.StringIO(r.text))
    except Exception as e:
        raise RuntimeError("Não foi possível localizar ou baixar o CSV de dados. "
                           "Ajuste o caminho local ou o RAW_URL.") from e

# Índice trimestral (Q-DEC)
df0["periodo_tri"] = df0["periodo_tri"].astype(str)
df_model = df0.set_index(pd.PeriodIndex(df0["periodo_tri"], freq="Q-DEC")).sort_index()
df_model.index.name = "periodo"

# Data no fim do trimestre
df_model["data"] = df_model.index.to_timestamp(how="end").normalize()

# Features do CAGED
df_model["caged_roll3"] = df_model["caged_saldo_tri"].rolling(3, min_periods=1).mean()  # causal e sem perder linhas
df_model["caged_roll3_asinh"] = np.arcsinh(df_model["caged_roll3"])

# Sazonalidade para ML (dummies trimestrais)
df_model["quarter"] = df_model.index.quarter
Q = pd.get_dummies(df_model["quarter"], prefix="Q", drop_first=False).astype("uint8")

# Mudança de regime e choque COVID
df_model["post_2021"] = (df_model["data"] >= "2021-01-01").astype("uint8")
df_model["d_covid"] = df_model.index.isin(pd.period_range("2020Q2","2021Q1", freq="Q-DEC")).astype(int)

# Concatenar dummies (sem dropna global)
df_model = pd.concat([df_model, Q], axis=1).copy()

print("Dimensão df_model:", df_model.shape)
display(df_model.head())

## Pré-processamento (checagens e política)
- Checagem de nulos e política de tratamento (sem vazamento temporal).
- Outliers estruturais COVID controlados via `d_covid`.

In [ ]:
# Checagem de nulos (formato horizontal)
display(df_model.isna().sum().to_frame("n_nulos").T)

# Gate opcional de consistência
assert df_model.isna().sum().sum() == 0, "Há NAs remanescentes no df_model."

**Conclusão da checagem de nulos.**  
A verificação indica **ausência de valores ausentes** nas variáveis utilizadas.
Assim, **não foi necessária imputação** na amostra atual.
As transformações do CAGED (`roll3`, `asinh`) não deixaram NAs remanescentes.
**Política preventiva:** caso surjam nulos em versões futuras, a imputação será aplicada **apenas nas exógenas**,
priorizando `ffill/bfill` calculados **no conjunto de treino** e, como *fallback*, **média sazonal por trimestre** (estimada no treino),
evitando **vazamento temporal**. A variável `d_covid` controla o **choque de 2020Q2–2021Q1**.

## EDA — decomposição e correlações (ACF/PACF)

**Objetivo.** Evidenciar tendência, sazonalidade trimestral (s=4) e persistência autocorrelativa,
para fundamentar a especificação sazonal e a necessidade de diferenciação.

In [ ]:
# Decomposição (PNAD e CAGED) — aditiva, período=4
fig = seasonal_decompose(df_model["tx_desocupacao"].asfreq("Q-DEC").to_timestamp(how="end"),
                         period=4, model="additive").plot()
plt.suptitle("Decomposição — tx_desocupacao", y=1.02)
plt.tight_layout(); plt.show(); plt.close()

fig = seasonal_decompose(df_model["caged_saldo_tri"].asfreq("Q-DEC").to_timestamp(how="end"),
                         period=4, model="additive").plot()
plt.suptitle("Decomposição — caged_saldo_tri", y=1.02)
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# ACF/PACF — PNAD (uma figura por gráfico)
y_all = df_model["tx_desocupacao"].asfreq("Q-DEC")
plt.figure()
plot_acf(y_all.dropna(), lags=16)
plt.title("ACF — tx_desocupacao")
plt.tight_layout(); plt.show(); plt.close()

plt.figure()
plot_pacf(y_all.dropna(), lags=16, method="ywm")
plt.title("PACF — tx_desocupacao")
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# ACF/PACF — CAGED nível e CAGED roll3/asinh
x_caged = df_model["caged_saldo_tri"].asfreq("Q-DEC")
x_roll3 = df_model["caged_roll3"].asfreq("Q-DEC")
x_asinh = df_model["caged_roll3_asinh"].asfreq("Q-DEC")

# Nível
plt.figure(); plot_acf(x_caged.dropna(), lags=16); plt.title("ACF — caged_saldo_tri"); plt.tight_layout(); plt.show(); plt.close()
plt.figure(); plot_pacf(x_caged.dropna(), lags=16, method="ywm"); plt.title("PACF — caged_saldo_tri"); plt.tight_layout(); plt.show(); plt.close()

# Roll3
plt.figure(); plot_acf(x_roll3.dropna(), lags=16); plt.title("ACF — caged_roll3"); plt.tight_layout(); plt.show(); plt.close()
plt.figure(); plot_pacf(x_roll3.dropna(), lags=16, method="ywm"); plt.title("PACF — caged_roll3"); plt.tight_layout(); plt.show(); plt.close()

# asinh(roll3)
plt.figure(); plot_acf(x_asinh.dropna(), lags=16); plt.title("ACF — asinh(caged_roll3)"); plt.tight_layout(); plt.show(); plt.close()
plt.figure(); plot_pacf(x_asinh.dropna(), lags=16, method="ywm"); plt.title("PACF — asinh(caged_roll3)"); plt.tight_layout(); plt.show(); plt.close()

**Leitura EDA (síntese).**  
- A PNAD exibe **não-estacionariedade em nível** (ACF com decaimento lento; PACF com pico em 1) e **sazonalidade s=4**.  
- O CAGED mostra **sazonalidade persistente** (picos em lags múltiplos de 4) e **alta variância**; `roll3` e `asinh` **reduzem variância e persistência** de curto prazo, mantendo o sinal sazonal — melhor para uso como exógena.

## Pipeline (Etapa 3)
1. **Coleta e construção do `df_model`** (Q‑DEC, variáveis, *features* derivadas).  
2. **Pré-processamento** (checagem de nulos/outliers e `d_covid`; padronização de exógenas **no treino**).  
3. **EDA** (decomposição, ACF/PACF) → **decisões** (s=4; d=1; exógenas transformadas).  
4. **Modelos base**: sNaive (baseline), **SARIMAX (CAGED)**, ablação **(+SELIC)** e ML opcional.  
5. **Validação base**: holdout + **CV rolling-origin**; **diagnósticos** dos resíduos.  
6. **Comparação e nota técnica**; limitações e próximos passos.

In [ ]:
# =======================
# Holdout e baseline sNaive
# =======================
TEST_LAST_N = 4

train_sar = df_model.iloc[:-TEST_LAST_N].copy()
test_sar  = df_model.iloc[-TEST_LAST_N:].copy()

y_tr = train_sar["tx_desocupacao"].asfreq("Q-DEC")
y_te = test_sar["tx_desocupacao"].asfreq("Q-DEC")

X_tr = train_sar[["caged_saldo_tri"]].asfreq("Q-DEC")
X_te = test_sar[["caged_saldo_tri"]].asfreq("Q-DEC")

# Padronização das exógenas usando média/desvio do TREINO
mu = X_tr.mean()
sd = X_tr.std(ddof=0).replace(0, 1.0)
X_tr_std = (X_tr - mu)/sd
X_te_std = (X_te - mu)/sd

def sNaive(y_train, h, s):
    last = y_train.iloc[-s:]
    reps = int(np.ceil(h/s))
    vals = np.tile(last.values, reps)[:h]
    return pd.Series(vals, index=y_te.index)

SEAS = 4
snaive = sNaive(y_tr, len(y_te), SEAS)

## Modelo base — SARIMAX (exógena = CAGED)
Especificação **(1,1,2) × (1,0,1,4)**, com `enforce_* = True`.  
Ajuste em duas fases (**Powell → L‑BFGS**), `cov_type="oim"`, sem `tol` deprecado.  
Se necessário, *fallback* relaxa restrições para obter `start_params` e reimpõe as restrições.

In [ ]:
ORDER = (1,1,2)
SEAS_ORDER = (1,0,1,4)

# Ajuste com restrições
model = SARIMAX(y_tr, exog=X_tr_std, order=ORDER, seasonal_order=SEAS_ORDER,
                enforce_stationarity=True, enforce_invertibility=True)

# 1ª fase: Powell para start
res0 = model.fit(method="powell", maxiter=400, disp=False)

# 2ª fase: L-BFGS
sarimax = model.fit(method="lbfgs", start_params=res0.params, maxiter=5000, disp=False, cov_type="oim")

# Fallback de convergência (caso necessário)
if hasattr(sarimax, "mle_retvals") and not sarimax.mle_retvals.get("converged", True):
    model_relax = SARIMAX(y_tr, exog=X_tr_std, order=ORDER, seasonal_order=SEAS_ORDER,
                          enforce_stationarity=False, enforce_invertibility=False)
    res_relax = model_relax.fit(method="lbfgs", maxiter=3000, disp=False, cov_type="oim")
    model_constr = SARIMAX(y_tr, exog=X_tr_std, order=ORDER, seasonal_order=SEAS_ORDER,
                           enforce_stationarity=True, enforce_invertibility=True)
    sarimax = model_constr.fit(method="lbfgs", start_params=res_relax.params, maxiter=5000, disp=False, cov_type="oim")

# Validade por raízes
def _min_root_abs(arr):
    if arr is None or len(arr)==0:
        return np.nan
    return float(np.min(np.abs(arr)))

mn_ar = _min_root_abs(getattr(sarimax, "arroots", np.array([])))
mn_ma = _min_root_abs(getattr(sarimax, "maroots", np.array([])))

print(">> Especificação final usada:", "ORDER", ORDER, "| SEAS", SEAS_ORDER)
print(f">> min|AR root|={mn_ar}, min|MA root|={mn_ma} (válido se ambos > 1)")

In [ ]:
# Forecast
fcst = sarimax.get_forecast(steps=len(y_te), exog=X_te_std)
pred_sar = fcst.predicted_mean
ci = fcst.conf_int(alpha=0.05)
ci_low = ci.iloc[:, 0].to_numpy()
ci_high = ci.iloc[:, 1].to_numpy()

# Métricas
def _rmse(a,b): a=np.asarray(a,float); b=np.asarray(b,float); return np.sqrt(np.mean((a-b)**2))
def _mape(a,b): a=np.asarray(a,float); b=np.asarray(b,float); m=a!=0; return (np.abs((a[m]-b[m])/np.maximum(np.abs(a[m]),1e-12))).mean()*100

y_true = np.asarray(y_te.values, float).ravel()
y_pred = np.asarray(pred_sar.values, float).ravel()

sarimax_metrics = pd.DataFrame({"MAE":[np.mean(np.abs(y_true-y_pred))],
                                "RMSE":[_rmse(y_true,y_pred)],
                                "MAPE (%)":[_mape(y_true,y_pred)]},
                               index=["SARIMAX (CAGED)"]).round(4)
display(sarimax_metrics)

rmse_snaive = _rmse(y_true, snaive.values)
print(f">> RMSE sNaive = {rmse_snaive:.3f} | Ganho vs sNaive = {100*(1 - sarimax_metrics['RMSE'].iloc[0]/max(rmse_snaive,1e-12)):.1f}%")

# Gráfico Real vs Pred + IC
t_idx = y_te.index.to_timestamp(how="end")
plt.figure()
plt.plot(t_idx, y_te.values, label="Real")
plt.plot(t_idx, pred_sar.values, label="SARIMAX")
plt.fill_between(t_idx, ci_low, ci_high, alpha=0.2, label="IC 95%")
plt.title("Real vs Pred — SARIMAX (baseline)")
plt.xlabel("Tempo"); plt.ylabel("Taxa de desocupação (p.p.)")
plt.legend(); plt.tight_layout(); plt.show(); plt.close()

### Diagnósticos dos resíduos (ACF, Ljung-Box, ARCH-LM, Jarque–Bera)

In [ ]:
resid = sarimax.resid.dropna()

# ACF e PACF (separados)
plt.figure(); plot_acf(resid.values, lags=16); plt.title("ACF — Resíduos (SARIMAX)"); plt.tight_layout(); plt.show(); plt.close()
plt.figure(); plot_pacf(resid.values, lags=16, method="ywm"); plt.title("PACF — Resíduos (SARIMAX)"); plt.tight_layout(); plt.show(); plt.close()

# Ljung-Box
lb = acorr_ljungbox(resid, lags=[1,4,8,12,16], return_df=True)
display(lb)

# Jarque-Bera (manual)
jb_stat, jb_p, skew, kurt = jarque_bera(resid)
print(f"Jarque-Bera: estat={jb_stat:.3f}, p={jb_p:.4f}, skew={skew:.3f}, kurt(excesso)={kurt:.3f}")

# ARCH-LM (usar nlags, não maxlag)
arch_stat, arch_p, _, _ = het_arch(resid, nlags=16)
print(f"ARCH-LM (lag 16): p-valor = {arch_p:.4f}  (ideal > 0.05)")

### Sumário do modelo (sem linhas de *Warnings*)

In [ ]:
smry = sarimax.summary()
try:
    et = getattr(smry, "extra_txt", None)
    if et:
        if isinstance(et, (list, tuple)):
            et_f = [t for t in et if "Covariance matrix" not in t]
            smry.extra_txt = "\n".join(et_f) if et_f else None
        elif isinstance(et, str):
            et_f = "\n".join([ln for ln in et.splitlines() if "Covariance matrix" not in ln])
            smry.extra_txt = et_f if et_f.strip() else None
except Exception:
    pass

print(smry.as_text())

## Validação base — Rolling-origin CV
Walk-forward (janela expansiva), **h = 1** e **h = 4**.

In [ ]:
from sklearn.metrics import mean_absolute_error

def _rmse_vec(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float); return np.sqrt(np.mean((a - b)**2))

def rolling_origin_cv(y, X, order=ORDER, seas_order=SEAS_ORDER,
                      initial=36, h=1, step=1, standardize=True):
    y = y.asfreq("Q-DEC"); X = X.asfreq("Q-DEC")
    n = len(y); assert n > initial + h, "Amostra insuficiente para CV."
    rows = []; fold = 0
    for t in range(initial, n - h + 1, step):
        fold += 1
        y_tr_f, y_te_f = y.iloc[:t],    y.iloc[t:t+h]
        X_tr_f, X_te_f = X.iloc[:t, :], X.iloc[t:t+h, :]

        if standardize:
            mu_f = X_tr_f.mean(); sd_f = X_tr_f.std(ddof=0).replace(0, 1.0)
            X_tr_, X_te_ = (X_tr_f - mu_f)/sd_f, (X_te_f - mu_f)/sd_f
        else:
            X_tr_, X_te_ = X_tr_f, X_te_f

        model_f = SARIMAX(y_tr_f, exog=X_tr_, order=order, seasonal_order=seas_order,
                          enforce_stationarity=True, enforce_invertibility=True)
        try:
            res_f = model_f.fit(method="lbfgs", disp=False, maxiter=1000, cov_type="oim")
            if hasattr(res_f, "mle_retvals") and not res_f.mle_retvals.get("converged", True):
                raise RuntimeError("not converged")
        except Exception:
            # Powell para start → L-BFGS
            try:
                res0_f = model_f.fit(method="powell", maxiter=300, disp=False)
                res_f  = model_f.fit(method="lbfgs", start_params=res0_f.params,
                                     disp=False, maxiter=1500, cov_type="oim")
            except Exception:
                relax = SARIMAX(y_tr_f, exog=X_tr_, order=order, seasonal_order=seas_order,
                                enforce_stationarity=False, enforce_invertibility=False
                               ).fit(method="lbfgs", disp=False, maxiter=800, cov_type="oim")
                res_f = SARIMAX(y_tr_f, exog=X_tr_, order=order, seasonal_order=seas_order,
                                enforce_stationarity=True, enforce_invertibility=True
                               ).fit(method="lbfgs", start_params=relax.params,
                                     disp=False, maxiter=1500, cov_type="oim")

        pred = res_f.get_forecast(steps=h, exog=X_te_).predicted_mean
        y_hat = np.asarray(pred.values, float).ravel()
        y_ref = np.asarray(y_te_f.values,  float).ravel()

        rows.append({
            "fold": fold,
            "inicio_treino": y_tr_f.index[0], "fim_treino": y_tr_f.index[-1],
            "inicio_teste": y_te_f.index[0],  "fim_teste": y_te_f.index[-1],
            "h": h,
            "MAE": mean_absolute_error(y_ref, y_hat),
            "RMSE": _rmse_vec(y_ref, y_hat),
        })

    cv = pd.DataFrame(rows)
    resumo = pd.DataFrame({
        "MAE (média ± dp)":  [f"{cv['MAE'].mean():.3f} ± {cv['MAE'].std(ddof=1):.3f}"],
        "RMSE (média ± dp)": [f"{cv['RMSE'].mean():.3f} ± {cv['RMSE'].std(ddof=1):.3f}"],
        "Folds": [len(cv)],
        "h": [h],
    }, index=[f"SARIMAX (exógenas = {list(X.columns)})"])
    return cv, resumo

y_all   = df_model["tx_desocupacao"]
X_caged = df_model[["caged_saldo_tri"]]

cv_h1, resumo_h1 = rolling_origin_cv(y_all, X_caged, initial=36, h=1, step=1)
cv_h4, resumo_h4 = rolling_origin_cv(y_all, X_caged, initial=36, h=4, step=1)

display(resumo_h1)
display(resumo_h4)

## Ablação — SARIMAX (CAGED) × SARIMAX (CAGED + SELIC)

In [ ]:
def _fit_predict(y_tr, X_tr, y_te, X_te, order=ORDER, seas=SEAS_ORDER):
    mu, sd = X_tr.mean(), X_tr.std(ddof=0).replace(0, 1.0)
    Xtr, Xte = (X_tr - mu)/sd, (X_te - mu)/sd
    mod = SARIMAX(y_tr, exog=Xtr, order=order, seasonal_order=seas,
                  enforce_stationarity=True, enforce_invertibility=True)
    try:
        res = mod.fit(method="lbfgs", disp=False, maxiter=2000, cov_type="oim")
        if hasattr(res, "mle_retvals") and not res.mle_retvals.get("converged", True):
            raise RuntimeError("not converged")
    except Exception:
        res0 = mod.fit(method="powell", maxiter=300, disp=False)
        res  = mod.fit(method="lbfgs", start_params=res0.params, disp=False, maxiter=3000, cov_type="oim")
    pred = res.get_forecast(steps=len(y_te), exog=Xte).predicted_mean
    return res, pred

# Baseline CAGED
res_base, pred_base = _fit_predict(y_tr, train_sar[["caged_saldo_tri"]], y_te, test_sar[["caged_saldo_tri"]])

# CAGED + SELIC (se existir)
if "selic_tri" in df_model.columns:
    res_both, pred_both = _fit_predict(y_tr, train_sar[["caged_saldo_tri","selic_tri"]],
                                       y_te,  test_sar[["caged_saldo_tri","selic_tri"]])
    def _rmse(a,b): a=np.asarray(a,float); b=np.asarray(b,float); return np.sqrt(np.mean((a-b)**2))
    y_true = np.asarray(y_te.values, float).ravel()
    m_base = {"MAE": float(np.mean(np.abs(y_true - np.asarray(pred_base.values, float).ravel()))),
              "RMSE": float(_rmse(y_true, np.asarray(pred_base.values, float).ravel()))}
    m_both = {"MAE": float(np.mean(np.abs(y_true - np.asarray(pred_both.values, float).ravel()))),
              "RMSE": float(_rmse(y_true, np.asarray(pred_both.values, float).ravel()))}
    comparativo = pd.DataFrame([m_base, m_both],
                               index=["SARIMAX (CAGED)", "SARIMAX (CAGED+SELIC)"]).round(4)
    display(comparativo)
else:
    print("Coluna 'selic_tri' não encontrada para a ablação CAGED+SELIC.")

## Baseline de ML (opcional) — LGBM (com *fallback* para GBR)

In [ ]:
ml_available = True
try:
    import lightgbm as lgb
except Exception:
    from sklearn.ensemble import GradientBoostingRegressor as GBR
    ml_available = False

# Features para ML (inclui dummies e regime; sem usar sazonalidade interna como no SARIMAX)
feats = [c for c in ["caged_saldo_tri","selic_tri","Q_1","Q_2","Q_3","Q_4","post_2021","d_covid"] if c in df_model.columns]
X_all = df_model[feats].copy()
y_all = df_model["tx_desocupacao"].copy()

X_tr_ml, X_te_ml = X_all.iloc[:-TEST_LAST_N], X_all.iloc[-TEST_LAST_N:]
y_tr_ml, y_te_ml = y_all.iloc[:-TEST_LAST_N], y_all.iloc[-TEST_LAST_N:]

# Padronização simples para ML (z-score usando treino)
mu_ml, sd_ml = X_tr_ml.mean(), X_tr_ml.std(ddof=0).replace(0,1.0)
X_tr_ml = (X_tr_ml - mu_ml)/sd_ml
X_te_ml = (X_te_ml - mu_ml)/sd_ml

if ml_available:
    dtrain = lgb.Dataset(X_tr_ml, label=y_tr_ml)
    params = dict(objective="regression", metric="l2", num_leaves=31, min_data_in_leaf=5,
                  learning_rate=0.1, feature_fraction=0.9, bagging_fraction=0.9, bagging_freq=0)
    model_ml = lgb.train(params, dtrain, num_boost_round=400)
    pred_ml = pd.Series(model_ml.predict(X_te_ml), index=y_te_ml.index, name="pred_ml")
else:
    model_ml = GBR(random_state=42)
    model_ml.fit(X_tr_ml, y_tr_ml)
    pred_ml = pd.Series(model_ml.predict(X_te_ml), index=y_te_ml.index, name="pred_ml")

def _rmse(a,b): a=np.asarray(a,float); b=np.asarray(b,float); return np.sqrt(np.mean((a-b)**2))
m_mae = float(np.mean(np.abs(y_te_ml.values - pred_ml.values)))
m_rmse = float(_rmse(y_te_ml.values, pred_ml.values))
mape = float(np.mean(np.abs((y_te_ml.values - pred_ml.values)/np.maximum(np.abs(y_te_ml.values),1e-12))))*100

ml_metrics = pd.DataFrame({"MAE":[m_mae], "RMSE":[m_rmse], "MAPE (%)":[mape]},
                          index=["LGBM" if ml_available else "GBR (fallback)"]).round(4)
display(ml_metrics)

# Comparação simples
try:
    comp = pd.concat([ml_metrics, sarimax_metrics])
    display(comp)
except Exception:
    pass

## Nota técnica — Modelo base (SARIMAX)

> Esta nota é gerada com os resultados calculados acima. Ajuste os textos conforme necessário.

In [ ]:
# Monta texto da nota técnica dinamicamente com os resultados atuais
try:
    mae = float(sarimax_metrics["MAE"].iloc[0])
    rmse = float(sarimax_metrics["RMSE"].iloc[0])
    mape = float(sarimax_metrics["MAPE (%)"].iloc[0])
    ganho = round(100*(1 - rmse/max(rmse_snaive,1e-12)), 1)

    lb_pvalues = acorr_ljungbox(sarimax.resid.dropna(), lags=[1,4,8,12,16], return_df=True)["lb_pvalue"].round(4).tolist()
    arch_p = float(het_arch(sarimax.resid.dropna(), nlags=16)[1])
    jb_stat, jb_p, skew, kurt = jarque_bera(sarimax.resid.dropna())

    nota = f'''
**Dados e split.** Série trimestral (Q-DEC), alvo `tx_desocupacao`. Exógena: `caged_saldo_tri` (baseline) e, em ablação, `selic_tri`.
Holdout de 4 trimestres.

**Pré-processamento.** Exógenas padronizadas com média/desvio do **treino**; transformações do CAGED (roll3/asinh) usadas na EDA;
`d_covid` (2020Q2–2021Q1) para choque estrutural. Sem NAs remanescentes.

**Especificação.** SARIMAX(1,1,2)×(1,0,1,4); `enforce_* = True`; ajuste **Powell → L-BFGS**, sem `tol` deprecado; `cov_type="oim"`.
**Validade (raízes).** min|AR|={mn_ar:.6f}, min|MA|={mn_ma:.6f} (>1) ⇒ estacionário (cond.) e invertível.

**Desempenho (holdout).** MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%.
sNaive RMSE={rmse_snaive:.3f}; **ganho vs sNaive** = {ganho}%.
    
**Diagnósticos.**
- **Ljung-Box (1,4,8,12,16)** p-values = {lb_pvalues} ⇒ sem autocorrelação remanescente.
- **ARCH-LM (16)** p={arch_p:.4f} ⇒ sem heterocedasticidade condicional.
- **Jarque-Bera** estat={jb_stat:.2f}, p={jb_p:.4f} ⇒ caudas pesadas; previsão não comprometida; usar `cov_type="oim"` (ou `robust_oim`) para inferência.

**Comparação.** SARIMAX supera o baseline sNaive e o ML (ver tabela). Ablação indica ganho com SELIC.

**Limitações e próximos passos.** Amostra trimestral curta e raízes próximas de 1 ⇒ cautela em horizontes longos.
Próximos passos: defasar/transformar CAGED (roll3/asinh), manter SELIC, dummies de choque e confirmar robustez em CV h=1 e h=4.
'''
    from IPython.display import Markdown, display as _display
    _display(Markdown(nota))
except Exception as e:
    print("Não foi possível montar a nota técnica automaticamente:", e)

---

> **Checklist Etapa 3**  
> ☑ EDA; ☑ Pré-processamento (nulos/outliers + `d_covid`); ☑ Modelo base (SARIMAX) + validação (holdout + CV);  
> ☑ Diagnósticos (ACF, Ljung‑Box, ARCH‑LM, JB); ☑ Comparação com baseline(s); ☑ Nota técnica e próximos passos.